In [ ]:
#Initial step: Setting up the environment
!pip install -U openai
!pip install pypdf
!pip install chromadb
!pip install -U langchain-community
!pip install -U langchain-openai
!pip install lark
!pip install trulens_eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.7 MB/s eta

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
# Step 1: Document loading
loader = PyPDFLoader("/content/acad_55.pdf")

In [ ]:
pages = loader.load()

In [ ]:
# Step 2: Splitting the documents into chuncks
#from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [ ]:
docs = text_splitter.split_documents(pages)
print("No. of Documents after split :",len(docs))
#print("Length of Notion DB : ",len(notion_db))

No. of Documents after split : 290


In [ ]:
#Step 3: Indexing  Storing the indexed documents in the Vector database
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from google.colab import userdata
import os
persist_directory = '/tmp/chroma/'
embedding = OpenAIEmbeddings(openai_api_key=userdata.get('OPENAI_API_KEY'))
#!rm -rf '/tmp/chroma/'  # remove old database files if any
splits = text_splitter.split_documents(docs)
os.makedirs(persist_directory, exist_ok=True)
!chmod -R 777 /tmp/chroma

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

290


In [ ]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [ ]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="Academic Handbook of IITH",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="Question Answers",
        type="integer",
    ),
]

In [ ]:
document_content_description = "Academic Handbook"

In [ ]:
llm = OpenAI(openai_api_key=userdata.get('OPENAI_API_KEY'),model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
#QA Starts from here
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = '/content/chroma/'
embedding = OpenAIEmbeddings(openai_api_key=userdata.get('OPENAI_API_KEY'))
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
from langchain.chat_models import ChatOpenAI
llm_name = "gpt-3.5-turbo"
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
from langchain.prompts import PromptTemplate
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

qa_chain_mr = RetrievalQA.from_chain_type(
    llm, retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

qa_chain_refine = RetrievalQA.from_chain_type(
    llm, retriever=vectordb.as_retriever(),
    chain_type="refine"
)

qa_chain_rerank = RetrievalQA.from_chain_type(
    llm, retriever=vectordb.as_retriever(),
    chain_type="map_rerank"
)

In [ ]:
my_questions = []
with open('iith_questions.txt', 'r') as file:
    for line in file:
        item = line.strip()
        my_questions.append(item)

In [ ]:
for question in my_questions:
  result1 = qa_chain({"query": question})
  result2 = qa_chain_mr({"query": question})
  result3 = qa_chain_refine({"query": question})
  result4 = qa_chain_rerank({"query": question})
  print("Question : "+question+"\n")
  print("Answer 1 : "+result1["result"]+"\n")
  print("Answer 2 : "+result2["result"]+"\n")
  print("Answer 3 : "+result3["result"]+"\n")
  print("Answer 4 : "+result4["result"]+"\n\n")
  print("-"*100)

In [ ]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

In [ ]:
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)

In [ ]:
from trulens_eval import TruLlama
context_selection = TruLlama.select_source_nodes().node.text

In [ ]:
import nest_asyncio

nest_asyncio.apply()
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

In [ ]:
import numpy as np
from trulens_eval import TruLlama
context_selection = TruLlama.select_source_nodes().node.text

f_qs_relevance = (
    Feedback(provider.qs_relevance, name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

In [ ]:
from trulens_eval.feedback import GroundTruthAgreement

f_groundedness = (
        Feedback(provider.groundedness_measure_with_cot_reasons,name="Groundedness" )
    .on(context_selection)
    .on_output()
)

In [ ]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [ ]:
with tru_recorder as recording:
    for question in my_questions:
        response = query_engine.query(question)

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=[])